In [7]:
# Install dependencies - OpenSloth + vLLM + Analysis setup
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install accelerate==1.7.0
!pip install triton==3.2.0 

!pip install unsloth==2025.5.7 unsloth-zoo==2025.5.8 --no-cache

!pip install opensloth==0.1.7 

!pip install TextStreamer

print("✅ TT-12 Dependencies installed:")
print("🚀 OpenSloth: Multi-GPU training")
print("🎯 vLLM: Precise inference") 
print("📊 Analysis libraries: scikit-learn, matplotlib, seaborn")

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 308.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 138.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 346.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 263.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 340.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 292.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 340.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: transformers

In [8]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-17 13:30:18.388049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758115818.734513      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758115818.832098      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# 1. Configuration and Data Setup

In [4]:
%%writefile constants.py
# Using base Qwen3 1.7B model from Kaggle input (no internet needed)
BASE_MODEL_PATH = "/kaggle/input/qwen3-1.7b-unsloth-bnb-4bit/gguf/default/1/qwen3_4bit"  # Update this path as needed
LORA_PATH = "qwen3_1.7b_opensloth_lora_validation/"  # OpenSloth LoRA output path for validation
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

# TT-12 Validation Parameters
TRAINING_DATA_PERCENTAGE = .3  # Controllable % of training data (0.1 = 10%, 1.0 = 100%)
USE_STRATIFIED_SAMPLING = True  # Maintain rule distribution when sampling

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

print("✅ Using Qwen3 1.7B model from local Kaggle input")
print(f"🎯 TT-12: OpenSloth training + vLLM inference with {TRAINING_DATA_PERCENTAGE*100:.0f}% of data")
print(f"📊 Stratified sampling: {USE_STRATIFIED_SAMPLING}")

Overwriting constants.py


In [5]:
%%writefile utils.py
import pandas as pd
from datasets import Dataset, load_from_disk
from constants import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT, TRAINING_DATA_PERCENTAGE, USE_STRATIFIED_SAMPLING, DATA_PATH
import random, numpy as np
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
import os
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_example_based_training_data(data_path):
    """
    TT-12: Create training data from examples (like test-time training)
    This trains the model on examples, not actual comments
    """
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    
    # Sample data if needed while maintaining rule distribution
    if TRAINING_DATA_PERCENTAGE < 1.0:
        if USE_STRATIFIED_SAMPLING:
            # Stratified sampling to maintain rule distribution
            train_dataset = train_dataset.groupby('rule', group_keys=False).apply(
                lambda x: x.sample(frac=TRAINING_DATA_PERCENTAGE, random_state=42)
            ).reset_index(drop=True)
            print(f"📊 Stratified sampling: {len(train_dataset)} samples ({TRAINING_DATA_PERCENTAGE*100:.0f}%)")
        else:
            # Simple random sampling
            train_dataset = train_dataset.sample(frac=TRAINING_DATA_PERCENTAGE, random_state=42).reset_index(drop=True)
            print(f"📊 Random sampling: {len(train_dataset)} samples ({TRAINING_DATA_PERCENTAGE*100:.0f}%)")
    
    print(f"📊 Training data size: {len(train_dataset)} samples")
    print(f"📊 Rule distribution: {train_dataset['rule'].value_counts().to_dict()}")
    
    flatten = []
    
    # Create training data from examples (similar to test-time training)
    for violation_type in ["positive", "negative"]:
        for i in range(1, 3):
            sub_dataset = train_dataset[["rule","subreddit",
                                        "positive_example_1","positive_example_2",
                                        "negative_example_1","negative_example_2"]].copy()

            if violation_type == "positive":
                # Use positive example as the "body" to classify
                body_col = f"positive_example_{i}"
                other_positive_col = f"positive_example_{3-i}"  # other positive
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                # negative_example randomly selected
                sub_dataset["negative_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["negative_example_1"],
                    sub_dataset["negative_example_2"]
                )
                sub_dataset["rule_violation"] = 1  # Positive examples violate rules

            else:  # violation_type == "negative"
                # Use negative example as the "body" to classify
                body_col = f"negative_example_{i}"
                other_negative_col = f"negative_example_{3-i}"
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                sub_dataset["positive_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["positive_example_1"],
                    sub_dataset["positive_example_2"]
                )
                sub_dataset["rule_violation"] = 0  # Negative examples don't violate rules

            # Drop original candidate columns
            sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                      "negative_example_1","negative_example_2"], inplace=True)

            flatten.append(sub_dataset)

    # Merge all DataFrames
    example_training_df = pd.concat(flatten, axis=0)
    example_training_df = example_training_df.drop_duplicates(ignore_index=True)
    
    print(f"📊 Example-based training dataset: {len(example_training_df)} samples")
    print(f"📊 Positive examples: {sum(example_training_df['rule_violation'] == 1)}")
    print(f"📊 Negative examples: {sum(example_training_df['rule_violation'] == 0)}")
    
    return example_training_df


def get_real_comment_validation_data(data_path):
    """
    TT-12: Get real comments with labels for validation
    This is what we actually want to predict
    """
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    
    # Use actual comments and their labels for validation
    validation_df = train_dataset[["body", "rule", "subreddit", "rule_violation",
                                  "positive_example_1","positive_example_2",
                                  "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example for prompts
    validation_df["positive_example"] = np.where(
        np.random.rand(len(validation_df)) < 0.5,
        validation_df["positive_example_1"],
        validation_df["positive_example_2"]
    )
    validation_df["negative_example"] = np.where(
        np.random.rand(len(validation_df)) < 0.5,
        validation_df["negative_example_1"],
        validation_df["negative_example_2"]
    )

    # Drop original candidate columns
    validation_df.drop(columns=["positive_example_1","positive_example_2",
                               "negative_example_1","negative_example_2"], inplace=True)
    
    print(f"📊 Real comment validation dataset: {len(validation_df)} samples")
    print(f"📊 Rule violations: {sum(validation_df['rule_violation'] == 1)} positive, {sum(validation_df['rule_violation'] == 0)} negative")
    
    return validation_df


def build_dataset_for_opensloth(dataframe, tokenizer):
    """Build dataset for OpenSloth training with proper text formatting"""
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)
    
    # OpenSloth uses a text field
    dataframe["text"] = dataframe.apply(lambda row: 
        tokenizer.apply_chat_template(
            [
                {"role": "user", "content": row["prompt"]},
                {"role": "assistant", "content": POSITIVE_ANSWER if row["rule_violation"] == 1 else NEGATIVE_ANSWER},
            ],
            tokenize=False,
            add_generation_prompt=False,
        ),
        axis=1
    )
    
    dataset = Dataset.from_pandas(dataframe[["text"]])
    return dataset


def build_validation_dataset(dataframe):
    """Build dataset for validation (keep labels for evaluation)"""
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)
    dataframe = dataframe[["prompt", "rule_violation"]]  # Keep true labels for evaluation
    dataset = Dataset.from_pandas(dataframe)
    return dataset

def cache_dataset(cache_path):
    if os.path.exists(cache_path):
        print(f"💾 Loading cached dataset from {cache_path}")
        return

    print(f"💾 Caching dataset to {cache_path}")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="/kaggle/input/qwen3-1.7b-unsloth-bnb-4bit/gguf/default/1/qwen3_4bit",
        max_seq_length=2048,
        load_in_4bit=True,
    )
    
    print(tokeinzer.chat_template)
    tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>\n<|im_start|>assistant\n' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + '<|im_end|>' }}{% endif %}{% endfor %}"
    
    
    train_df = get_example_based_training_data(DATA_PATH)
    dataset = build_dataset_for_opensloth(train_df, tokenizer)
    
    # This is a bit of a hack. SFTTrainer is used for its dataset processing
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        args=SFTConfig(
            dataset_text_field="text",
            max_seq_length=2048,
            dataset_num_proc=2,
            packing=True,
        ),
    )
    
    trainer.train_dataset.save_to_disk(cache_path)
    print(f"✅ Dataset cached to {cache_path}")

def get_cached_dataset(cache_path):
    if not os.path.exists(cache_path):
        raise RuntimeError("Dataset cache not found. Please run the caching step first.")
    return load_from_disk(cache_path)


Writing utils.py


In [10]:
from transformers import TextStreamer

In [12]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/kaggle/input/qwen3-1.7b-unsloth-bnb-4bit/gguf/default/1/qwen3_4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)
print(tokenizer.chat_template)

==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_cal

In [13]:
from utils import *
train_df = get_example_based_training_data(DATA_PATH)
dataset = build_dataset_for_opensloth(train_df, tokenizer)



✅ Using Qwen3 1.7B model from local Kaggle input
🎯 TT-12: OpenSloth training + vLLM inference with 30% of data
📊 Stratified sampling: True
📊 Stratified sampling: 609 samples (30%)
📊 Training data size: 609 samples
📊 Rule distribution: {'No legal advice: Do not offer or request legal advice.': 305, 'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.': 304}
📊 Example-based training dataset: 2435 samples
📊 Positive examples: 1218
📊 Negative examples: 1217


In [15]:
print(dataset[9])

{'text': '<|im_start|>user\n\nYou are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.\n\nSubreddit: r/The_Donald\nRule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.\nExamples:\n1) Free paypal cards here!! https://www.pointsprizes.com/ref/13226\nAnswer: Yes\n\n2) **HD** Stream [English 1](http://www.zifootball.us/p/channel1.html) | [English 2](http://www.zifootball.us/p/channel2.html) | Ad Overlays: 4  | Mobile : Yes\n\nUpvote if you like the streams\nAnswer: No\n\n---\nComment: bad hookers waiting for you here http://cbvidfree.com/?id44838\nAnswer:<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nYes<|im_end|>\n'}


In [33]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer)
inputs = tokenizer(dataset[666]['text'], return_tensors="pt").to("cuda")

output = model.generate( **inputs ,streamer = text_streamer, max_new_tokens = 512)

<|im_start|>user

You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.

Subreddit: r/news
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Examples:
1) [SANDERS SUPPORTERS DON'T WORRY. LENDING TREE IS WILLING TO MAKE LOANS OF UP TO 32000 DOLLARS SO YOU CAN DONATE TO YOUR LORD AND SAVIOR](https://www.lendingtree.com/personal-loans-index)
Answer: Yes

2) **HD** [ German Stream ](http://www.sportonline-tv.com/stream3.php) | Mobile : Yes
Answer: No

---
Comment: 39 http://MySexFind.com - find girl for sex now 39!
Answer:<|im_end|>
<|im_start|>assistant
<think>

</think>

Yes<|im_end|>
<|endoftext|>


In [34]:

# Decode the full output to readable text
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n" + "="*50)
print("🎯 Full Decoded Output (including thinking):")
print(decoded_output)
print("="*512)



🎯 Full Decoded Output (including thinking):
user

You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.

Subreddit: r/news
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Examples:
1) [SANDERS SUPPORTERS DON'T WORRY. LENDING TREE IS WILLING TO MAKE LOANS OF UP TO 32000 DOLLARS SO YOU CAN DONATE TO YOUR LORD AND SAVIOR](https://www.lendingtree.com/personal-loans-index)
Answer: Yes

2) **HD** [ German Stream ](http://www.sportonline-tv.com/stream3.php) | Mobile : Yes
Answer: No

---
Comment: 39 http://MySexFind.com - find girl for sex now 39!
Answer:
assistant
<think>

</think>

Yes



Trying Thinking

In [51]:

# Thinking prompt (replace with your dataset[0]['text'] if needed)
thinking_prompt = """You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule.

First, think step-by-step about the comment and the rule.
Then, conclude with your Single final answer: 'Answer: Yes' or 'Answer: No'.Full stop after answer.Say nothing after giving out the answer.

Subreddit: r/news
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Examples:
1) [SANDERS SUPPORTERS DON'T WORRY. LENDING TREE IS WILLING TO MAKE LOANS OF UP TO 32000 DOLLARS SO YOU CAN DONATE TO YOUR LORD AND SAVIOR](https://www.lendingtree.com/personal-loans-index)
Answer: Yes

2) **HD** [ German Stream ](http://www.sportonline-tv.com/stream3.php) | Mobile : Yes
Answer: No

---
Comment: 39 http://MySexFind.com - find girl for sex now 39!

"""
messages = [
    {"role": "user", "content": thinking_prompt},
    {"role": "assistant", "content": "<think>"}
]
thinking_input = tokenizer.apply_chat_template(messages, tokenize=False)
# Tokenize the input
inputs = tokenizer(thinking_prompt, return_tensors="pt").to("cuda")

# Generate with Qwen Thinking Mode settings
text_streamer = TextStreamer(tokenizer)
output = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=500,  # Allow room for thinking
    temperature=0.6,     # Qwen Thinking Mode: Balanced creativity
    top_p=0.95,          # Qwen Thinking Mode: Diverse selection
    top_k=20,            # Qwen Thinking Mode: Limit to top 20 tokens
    min_p=0.0,           # Qwen Thinking Mode: No minimum probability threshold
    do_sample=True,      # Enable sampling for creativity
    #stop=["\n"],         # Stop after newline to prevent repetition
    pad_token_id=tokenizer.eos_token_id,
)

# Decode the full output to readable text
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n" + "="*500)
print("🎯 Full Decoded Output (with thinking):")
print(decoded_output)
print("="*500)

# Optional: Extract just the new part (after the prompt)
new_part = decoded_output[len(thinking_prompt):].strip()
print("\n🎯 New Generated Part:")
print(new_part)

You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule.

First, think step-by-step about the comment and the rule.
Then, conclude with your Single final answer: 'Answer: Yes' or 'Answer: No'.Full stop after answer.Say nothing after giving out the answer.

Subreddit: r/news
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Examples:
1) [SANDERS SUPPORTERS DON'T WORRY. LENDING TREE IS WILLING TO MAKE LOANS OF UP TO 32000 DOLLARS SO YOU CAN DONATE TO YOUR LORD AND SAVIOR](https://www.lendingtree.com/personal-loans-index)
Answer: Yes

2) **HD** [ German Stream ](http://www.sportonline-tv.com/stream3.php) | Mobile : Yes
Answer: No

---
Comment: 39 http://MySexFind.com - find girl for sex now 39!

Answer:
Step 1: Identify the key elements in the comment.
The comment starts with "39 http://MySexFind.com - find girl for sex now 39!" which includes a URL and the phrase "find girl 

KeyboardInterrupt: 

In [ ]:
%%writefile train_opensloth.py
from opensloth.opensloth_config import (
    FastModelArgs,
    LoraArgs,
    OpenSlothConfig,
    TrainingArguments,
)
from opensloth.scripts.opensloth_sft_trainer import run_mp_training, setup_envs
from constants import BASE_MODEL_PATH, LORA_PATH
import torch

# OpenSloth Configuration for 2 GPUs
GLOBAL_BZ = 16
DEVICES = [i for i in range(torch.cuda.device_count())]
BZ = 2  # Batch size per device

opensloth_config = OpenSlothConfig(
    data_cache_path="data/cache_qwen3_dataset_for_opensloth/",
    devices=DEVICES,
    fast_model_args=FastModelArgs(
        model_name=BASE_MODEL_PATH,
        max_seq_length=2048,
        load_in_4bit=True,
        local_files_only=True,
        trust_remote_code=True,
    ),
    lora_args=LoraArgs(
        r=16,
        lora_alpha=16,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_dropout=0,
        bias="none",
        use_rslora=False,
    ),
    sequence_packing=True,
)

training_config = TrainingArguments(
    output_dir=LORA_PATH,
    per_device_train_batch_size=BZ,
    gradient_accumulation_steps=GLOBAL_BZ // (len(DEVICES) * BZ) if DEVICES else 1,
    learning_rate=1e-5,
    logging_steps=1,
    num_train_epochs=1, # Using max_steps instead
    max_steps=60,
    lr_scheduler_type="linear",
    warmup_steps=5,
    save_total_limit=2,
    save_strategy="steps",
    save_steps=20,
    weight_decay=0.01,
    optim="adamw_8bit",
    seed=3407,
    report_to="none",
)


if __name__ == "__main__":
    import os

    print(f"Global batch size: {len(DEVICES) * BZ * training_config.gradient_accumulation_steps}")
    print(f"Gradient accumulation steps: {training_config.gradient_accumulation_steps}")

    setup_envs(opensloth_config, training_config)
    run_mp_training(opensloth_config.devices, opensloth_config, training_config)
    
    print(f"✅ OpenSloth training completed! LoRA adapters saved to: {LORA_PATH}")
    print("🎯 Ready for vLLM inference!")


# 🎯 2x T4 GPU Optimization Guide

## ⚡ **Multi-GPU Configuration for TT-11**

### **Your Setup: 2x T4 (28GB Total VRAM)**
- **GPU 0**: ~14GB VRAM
- **GPU 1**: ~14GB VRAM
- **Total**: 28GB available for training

### **Optimizations Applied:**

#### **1. Model Distribution**
```python
device_map="auto"  # Automatic distribution across GPUs
max_memory={0: "13GB", 1: "13GB"}  # Reserve 1GB per GPU for operations
```

#### **2. Batch Size Scaling**
```python
per_device_train_batch_size=4,  # 4 samples per GPU (8 total)
gradient_accumulation_steps=2,  # Effective batch = 4*2*2 = 16
```

#### **3. Memory Optimizations**
```python
load_in_4bit=True,              # 4-bit quantization saves ~75% memory
use_gradient_checkpointing=True, # Trade compute for memory
dataloader_pin_memory=False,     # Let Unsloth handle memory
```

#### **4. Multi-GPU Training**
```python
dataloader_num_workers=4,        # Parallel data loading
ddp_find_unused_parameters=False, # DDP optimization
ddp_broadcast_buffers=False,     # Reduce communication
```

### **Expected Performance:**
- **Training Speed**: 3x-6x faster than single GPU
- **Memory Usage**: ~12-13GB per GPU
- **Effective Batch**: 16 samples (vs 4 on single GPU)
- **Total Time**: 5-8 minutes for full training

### **Troubleshooting 2x T4:**

#### **If you get OOM (Out of Memory):**
```python
# Reduce batch size
per_device_train_batch_size=2,   # 2 per GPU instead of 4
gradient_accumulation_steps=4,   # Keep effective batch size

# Or reduce sequence length
max_seq_length=1024,             # Shorter sequences
```

#### **If training is slower than expected:**
```python
# Check GPU utilization
nvidia-smi  # Should show ~90%+ on both GPUs

# Increase batch size if memory allows
per_device_train_batch_size=6,   # Try larger batches
```

#### **Memory Distribution Check:**
```python
print(f"Available GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_properties(i).total_memory // 1024**3}GB")
```

In [ ]:
%%writefile validation_vllm.py
import os
os.environ["VLLM_USE_V1"] = "0"

import vllm
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, 
                           roc_auc_score, confusion_matrix, classification_report, roc_curve)
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from vllm.lora.request import LoRARequest
from utils import build_validation_dataset, get_real_comment_validation_data
from constants import BASE_MODEL_PATH, LORA_PATH, DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER


def run_validation_vllm():
    """Run validation using OpenSloth-trained model with vLLM for precise AUC"""
    
    # Get real comment validation data
    val_df = get_real_comment_validation_data(DATA_PATH)
    val_dataset = build_validation_dataset(val_df)
    
    print(f"🔍 Running validation on {len(val_dataset)} real comments")
    
    # 🎯 VLLM: Initialize with OpenSloth LoRA support for precise probabilities
    llm = vllm.LLM(
        BASE_MODEL_PATH,
        tensor_parallel_size=torch.cuda.device_count(),
        gpu_memory_utilization=0.90,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=512,
        disable_log_stats=True,
        enable_prefix_caching=True,
        enable_lora=True,
        max_lora_rank=64,
        local_files_only=True,
    )

    tokenizer = llm.get_tokenizer()

    texts = val_dataset["prompt"]
    true_labels = val_dataset["rule_violation"]

    # 🎯 VLLM: Generate with OpenSloth LoRA for most accurate probabilities
    outputs = llm.generate(
        texts,
        vllm.SamplingParams(
            skip_special_tokens=True,
            max_tokens=1,
            logprobs=20,
        ),
        use_tqdm=True,
        lora_request=LoRARequest("opensloth_lora", 1, LORA_PATH)  # Load OpenSloth LoRA
    )

    # Extract predictions and probabilities with vLLM precision
    predictions = []
    probabilities = []
    
    yes_token_id = tokenizer.convert_tokens_to_ids("Yes")
    no_token_id = tokenizer.convert_tokens_to_ids("No")
    
    for out in outputs:
        log_probs = out.outputs[0].logprobs[0]
        
        log_prob_yes = log_probs.get(yes_token_id)
        log_prob_no = log_probs.get(no_token_id)
        
        if log_prob_yes is not None and log_prob_no is not None:
            if log_prob_yes.logprob > log_prob_no.logprob:
                predictions.append(1)
            else:
                predictions.append(0)
            
            exp_pos = np.exp(log_prob_yes.logprob)
            exp_neg = np.exp(log_prob_no.logprob)
            prob_positive = exp_pos / (exp_pos + exp_neg)
            probabilities.append(prob_positive)
        else:
            predictions.append(0)
            probabilities.append(0.5)

    return true_labels, predictions, probabilities, val_df


def calculate_and_display_metrics(true_labels, predictions, probabilities):
    """Calculate comprehensive metrics and display results"""
    
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    auc = roc_auc_score(true_labels, probabilities)
    
    print("=" * 60)
    print("📊 TT-12 VALIDATION RESULTS (OpenSloth + vLLM)")
    print("=" * 60)
    print(f"🎯 Accuracy:  {accuracy:.4f}")
    print(f"🎯 F1 Score:  {f1:.4f}")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🎯 Recall:    {recall:.4f}")
    print(f"🎯 AUC Score: {auc:.4f} (High-precision vLLM)")
    print("=" * 60)
    
    cm = confusion_matrix(true_labels, predictions)
    print("\n📈 Confusion Matrix:")
    print(f"True Negative: {cm[0,0]:4d} | False Positive: {cm[0,1]:4d}")
    print(f"False Negative: {cm[1,0]:4d} | True Positive:  {cm[1,1]:4d}")
    
    print("\n📋 Classification Report:")
    print(classification_report(true_labels, predictions, target_names=['No Violation', 'Violation']))
    
    return {
        'accuracy': accuracy, 'f1': f1, 'precision': precision,
        'recall': recall, 'auc': auc, 'confusion_matrix': cm
    }


def create_visualizations(true_labels, predictions, probabilities, metrics):
    """Create comprehensive visualizations"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('TT-12: OpenSloth Training + vLLM Validation Results', fontsize=16, fontweight='bold')
    
    cm = metrics['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
                xticklabels=['No Violation', 'Violation'],
                yticklabels=['No Violation', 'Violation'])
    axes[0,0].set_title('Confusion Matrix')
    axes[0,0].set_xlabel('Predicted')
    axes[0,0].set_ylabel('Actual')
    
    fpr, tpr, _ = roc_curve(true_labels, probabilities)
    axes[0,1].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {metrics["auc"]:.3f})')
    axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[0,1].set_xlabel('False Positive Rate')
    axes[0,1].set_ylabel('True Positive Rate')
    axes[0,1].set_title('ROC Curve (vLLM High-Precision)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    pos_probs = [p for p, t in zip(probabilities, true_labels) if t == 1]
    neg_probs = [p for p, t in zip(probabilities, true_labels) if t == 0]
    
    axes[1,0].hist(neg_probs, bins=30, alpha=0.7, label='No Violation', color='blue', density=True)
    axes[1,0].hist(pos_probs, bins=30, alpha=0.7, label='Violation', color='red', density=True)
    axes[1,0].set_xlabel('Predicted Probability (vLLM Precision)')
    axes[1,0].set_ylabel('Density')
    axes[1,0].set_title('Probability Distribution by True Label')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    metric_names = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    metric_values = [metrics[k] for k in ['accuracy', 'f1', 'precision', 'recall', 'auc']]
    
    bars = axes[1,1].bar(metric_names, metric_values, color=['skyblue', 'lightgreen', 'orange', 'pink', 'gold'])
    axes[1,1].set_ylabel('Score')
    axes[1,1].set_title('Performance Metrics (OpenSloth + vLLM)')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].grid(True, alpha=0.3, axis='y')
    
    for bar, value in zip(bars, metric_values):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/tt12_validation_results.png', dpi=300, bbox_inches='tight')
    plt.show()


def analyze_by_rule(true_labels, predictions, probabilities, val_df):
    """Analyze performance by rule type"""
    
    analysis_df = val_df.copy()
    analysis_df['predictions'] = predictions
    analysis_df['probabilities'] = probabilities
    
    print("\n📊 PERFORMANCE BY RULE (vLLM High-Precision AUC):")
    print("=" * 60)
    
    rule_metrics = []
    for rule in analysis_df['rule'].unique():
        rule_data = analysis_df[analysis_df['rule'] == rule]
        
        rule_true = rule_data['rule_violation'].values
        rule_pred = rule_data['predictions'].values
        rule_prob = rule_data['probabilities'].values
        
        rule_auc = roc_auc_score(rule_true, rule_prob) if len(np.unique(rule_true)) > 1 else np.nan
        rule_acc = accuracy_score(rule_true, rule_pred)
        rule_f1 = f1_score(rule_true, rule_pred) if len(np.unique(rule_true)) > 1 else np.nan
        
        print(f"Rule: {rule}\n  Samples: {len(rule_data)}\n  Accuracy: {rule_acc:.3f}\n  F1 Score: {rule_f1:.3f}\n  AUC Score: {rule_auc:.3f}\n")
        
        rule_metrics.append({'rule': rule, 'samples': len(rule_data), 'accuracy': rule_acc, 'f1': rule_f1, 'auc': rule_auc})
    
    analysis_df.to_csv('/kaggle/working/tt12_detailed_results.csv', index=False)
    pd.DataFrame(rule_metrics).to_csv('/kaggle/working/tt12_rule_metrics.csv', index=False)
    
    return rule_metrics


def main():
    print("🔬 TT-12: OpenSloth Training + vLLM Validation")
    print("🚀 Multi-GPU training + High-precision inference!")
    print("=" * 70)
    
    true_labels, predictions, probabilities, val_df = run_validation_vllm()
    metrics = calculate_and_display_metrics(true_labels, predictions, probabilities)
    create_visualizations(true_labels, predictions, probabilities, metrics)
    analyze_by_rule(true_labels, predictions, probabilities, val_df)
    
    print("✅ TT-12 Validation completed!")
    print("📈 Visualizations saved: /kaggle/working/tt12_validation_results.png")
    print("📊 Detailed results: /kaggle/working/tt12_detailed_results.csv")
    print("📋 Rule metrics: /kaggle/working/tt12_rule_metrics.csv")
    
if __name__ == "__main__":
    main()


In [ ]:
%%writefile validation_transformers.py
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, 
                           roc_auc_score, confusion_matrix, classification_report, roc_curve)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from tqdm import tqdm
from utils import build_validation_dataset, get_real_comment_validation_data
from constants import BASE_MODEL_PATH, LORA_PATH, DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER


def run_validation_transformers():
    """Run validation using standard transformers with OpenSloth LoRA - Universal compatibility"""
    
    val_df = get_real_comment_validation_data(DATA_PATH)
    val_dataset = build_validation_dataset(val_df)
    
    print(f"🔍 Running validation on {len(val_dataset)} real comments (Transformers)")
    
    print("📥 Loading base model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, local_files_only=True)
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        local_files_only=True,
    )
    
    print("🔗 Loading OpenSloth LoRA adapters...")
    model = PeftModel.from_pretrained(model, LORA_PATH)
    model = model.merge_and_unload()
    model.eval()
    
    yes_token_id = tokenizer.encode("Yes", add_special_tokens=False)[0]
    no_token_id = tokenizer.encode("No", add_special_tokens=False)[0]
    
    print(f"🎯 Token IDs: Yes={yes_token_id}, No={no_token_id}")
    
    texts = val_dataset["prompt"]
    true_labels = val_dataset["rule_violation"]
    
    predictions, probabilities = [], []
    batch_size = 8
    
    print("🚀 Running inference...")
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
            next_token_logits = outputs.logits[:, -1, :]
            
            yes_logits = next_token_logits[:, yes_token_id]
            no_logits = next_token_logits[:, no_token_id]
            
            combined_logits = torch.stack([no_logits, yes_logits], dim=1)
            probs = torch.softmax(combined_logits, dim=1)
            
            predictions.extend(torch.argmax(probs, dim=1).cpu().tolist())
            probabilities.extend(probs[:, 1].cpu().tolist())
    
    print("✅ Inference completed!")
    return true_labels, predictions, probabilities, val_df


def calculate_and_display_metrics(true_labels, predictions, probabilities):
    """Calculate comprehensive metrics and display results"""
    
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    auc = roc_auc_score(true_labels, probabilities)
    
    print("=" * 60)
    print("📊 TT-12 VALIDATION RESULTS (OpenSloth + Transformers)")
    print("=" * 60)
    print(f"🎯 Accuracy:  {accuracy:.4f}")
    print(f"🎯 F1 Score:  {f1:.4f}")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🎯 Recall:    {recall:.4f}")
    print(f"🎯 AUC Score: {auc:.4f} (Standard Transformers)")
    print("=" * 60)
    
    cm = confusion_matrix(true_labels, predictions)
    print("\n📈 Confusion Matrix:")
    print(f"True Negative: {cm[0,0]:4d} | False Positive: {cm[0,1]:4d}")
    print(f"False Negative: {cm[1,0]:4d} | True Positive:  {cm[1,1]:4d}")
    
    print("\n📋 Classification Report:")
    print(classification_report(true_labels, predictions, target_names=['No Violation', 'Violation']))
    
    return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall, 'auc': auc, 'confusion_matrix': cm}


def create_visualizations(true_labels, predictions, probabilities, metrics):
    """Create comprehensive visualizations"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('TT-12: OpenSloth Training + Transformers Validation Results', fontsize=16, fontweight='bold')
    
    cm = metrics['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
                xticklabels=['No Violation', 'Violation'],
                yticklabels=['No Violation', 'Violation'])
    axes[0,0].set_title('Confusion Matrix')
    axes[0,0].set_xlabel('Predicted')
    axes[0,0].set_ylabel('Actual')
    
    fpr, tpr, _ = roc_curve(true_labels, probabilities)
    axes[0,1].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {metrics["auc"]:.3f})')
    axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[0,1].set_xlabel('False Positive Rate')
    axes[0,1].set_ylabel('True Positive Rate')
    axes[0,1].set_title('ROC Curve (Transformers)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    pos_probs = [p for p, t in zip(probabilities, true_labels) if t == 1]
    neg_probs = [p for p, t in zip(probabilities, true_labels) if t == 0]
    
    axes[1,0].hist(neg_probs, bins=30, alpha=0.7, label='No Violation', color='blue', density=True)
    axes[1,0].hist(pos_probs, bins=30, alpha=0.7, label='Violation', color='red', density=True)
    axes[1,0].set_xlabel('Predicted Probability (Transformers)')
    axes[1,0].set_ylabel('Density')
    axes[1,0].set_title('Probability Distribution by True Label')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    metric_names = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC']
    metric_values = [metrics[k] for k in ['accuracy', 'f1', 'precision', 'recall', 'auc']]
    
    bars = axes[1,1].bar(metric_names, metric_values, color=['skyblue', 'lightgreen', 'orange', 'pink', 'gold'])
    axes[1,1].set_ylabel('Score')
    axes[1,1].set_title('Performance Metrics (OpenSloth + Transformers)')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].grid(True, alpha=0.3, axis='y')
    
    for bar, value in zip(bars, metric_values):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/tt12_transformers_validation_results.png', dpi=300, bbox_inches='tight')
    plt.show()


def analyze_by_rule(true_labels, predictions, probabilities, val_df):
    """Analyze performance by rule type"""
    
    analysis_df = val_df.copy()
    analysis_df['predictions'] = predictions
    analysis_df['probabilities'] = probabilities
    
    print("\n📊 PERFORMANCE BY RULE (Transformers):")
    print("=" * 60)
    
    rule_metrics = []
    for rule in analysis_df['rule'].unique():
        rule_data = analysis_df[analysis_df['rule'] == rule]
        
        rule_true = rule_data['rule_violation'].values
        rule_pred = rule_data['predictions'].values
        rule_prob = rule_data['probabilities'].values
        
        rule_auc = roc_auc_score(rule_true, rule_prob) if len(np.unique(rule_true)) > 1 else np.nan
        rule_acc = accuracy_score(rule_true, rule_pred)
        rule_f1 = f1_score(rule_true, rule_pred) if len(np.unique(rule_true)) > 1 else np.nan
        
        print(f"Rule: {rule}\n  Samples: {len(rule_data)}\n  Accuracy: {rule_acc:.3f}\n  F1 Score: {rule_f1:.3f}\n  AUC Score: {rule_auc:.3f}\n")
        
        rule_metrics.append({'rule': rule, 'samples': len(rule_data), 'accuracy': rule_acc, 'f1': rule_f1, 'auc': rule_auc})
    
    analysis_df.to_csv('/kaggle/working/tt12_transformers_detailed_results.csv', index=False)
    pd.DataFrame(rule_metrics).to_csv('/kaggle/working/tt12_transformers_rule_metrics.csv', index=False)
    
    return rule_metrics


def main():
    print("🔬 TT-12: OpenSloth Training + Transformers Validation")
    print("🚀 Multi-GPU training + Universal compatibility!")
    print("=" * 70)
    
    true_labels, predictions, probabilities, val_df = run_validation_transformers()
    metrics = calculate_and_display_metrics(true_labels, predictions, probabilities)
    create_visualizations(true_labels, predictions, probabilities, metrics)
    analyze_by_rule(true_labels, predictions, probabilities, val_df)
    
    print("✅ TT-12 Transformers Validation completed!")
    print("📈 Visualizations saved: /kaggle/working/tt12_transformers_validation_results.png")
    print("📊 Detailed results: /kaggle/working/tt12_transformers_detailed_results.csv")
    print("📋 Rule metrics: /kaggle/working/tt12_transformers_rule_metrics.csv")

if __name__ == "__main__":
    main()


In [ ]:
from utils import cache_dataset
cache_dataset(cache_path="data/cache_qwen3_dataset_for_opensloth/")

In [ ]:
!python train_opensloth.py

# Appendix: View Results

In [ ]:
#@title **Run Validation**
#@markdown Choose your validation method:
VALIDATION_METHOD = "vLLM" #@param ["vLLM", "Transformers"]

if VALIDATION_METHOD == "vLLM":
    print("🚀 Running vLLM validation for maximum speed and precision...")
    !python validation_vllm.py
else:
    print("⚙️ Running Transformers validation for universal compatibility...")
    !python validation_transformers.py

In [ ]:
from IPython.display import Image, display

# Display the validation results image
if VALIDATION_METHOD == "vLLM":
    display(Image(filename='/kaggle/working/tt12_validation_results.png'))
else:
    display(Image(filename='/kaggle/working/tt12_transformers_validation_results.png'))

In [ ]:
import pandas as pd

# Display the detailed results CSV
if VALIDATION_METHOD == "vLLM":
    df_detailed = pd.read_csv('/kaggle/working/tt12_detailed_results.csv')
else:
    df_detailed = pd.read_csv('/kaggle/working/tt12_transformers_detailed_results.csv')

df_detailed.head()